In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import keras
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.layers import Dense, Dropout, Flatten


Using TensorFlow backend.


In [2]:
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

[[22. 28.]
 [49. 64.]]


In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8142322328032889261
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3171064217
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14221653461441551838
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [ ]:
labels  = pd.read_csv('labels.csv')

In [ ]:
labels['breed'].nunique()

120

In [ ]:
labels['breed'].value_counts()

scottish_deerhound                126
maltese_dog                       117
afghan_hound                      116
entlebucher                       115
bernese_mountain_dog              114
shih-tzu                          112
pomeranian                        111
great_pyrenees                    111
basenji                           110
samoyed                           109
airedale                          107
tibetan_terrier                   107
leonberg                          106
cairn                             106
japanese_spaniel                  105
beagle                            105
australian_terrier                102
miniature_pinscher                102
blenheim_spaniel                  102
irish_wolfhound                   101
saluki                             99
lakeland_terrier                   99
papillon                           96
whippet                            95
norwegian_elkhound                 95
siberian_husky                     95
pug         

In [ ]:
import cv2
import os
im_size = 200
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            #images.append(img)
            images.append(cv2.resize(img, (im_size, im_size)))
    return images

In [ ]:
X_train = []
y_train = []

In [ ]:
X_test = []

In [ ]:
from tqdm import tqdm
i = 0
for f, breed in tqdm(labels.values):
    img = cv2.imread('D:\\ML\\Dog_Breed\\train\\{}.jpg'.format(f))
    #label = one_hot_labels[i]
    X_train.append(cv2.resize(img, (im_size, im_size)))
    #y_train.append(label)
    i += 1

100%|████████████████████████████████████████████████████████████████████████████| 10222/10222 [03:38<00:00, 46.73it/s]


In [ ]:
len(X_train)

10222

In [ ]:
X_train[0].shape

(200, 200, 3)

In [ ]:
X_train = np.array(X_train, np.float32)

In [ ]:
targets_series = pd.Series(labels['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)

#y_train = np.array(y_train, np.uint8)

In [ ]:
one_hot_labels = np.asarray(one_hot) 

In [ ]:
print(one_hot_labels)

In [ ]:
y_train = one_hot_labels

In [ ]:
y_train[0]

In [ ]:
X = X_train
y = y_train

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
predictions = Dense(120, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
model.summary()

In [ ]:
model.fit(X, y, epochs=300, verbose=1)

In [ ]:
test_labels = pd.read_csv('sample_submission.csv')

In [ ]:
test_labels

In [ ]:
for f in tqdm(test_labels['id'].values):
    img = cv2.imread('D:\\ML\\Dog_Breed\\test\\{}.jpg'.format(f))
    #label = one_hot_labels[i]
    X_test.append(cv2.resize(img, (im_size, im_size)))
    #y_train.append(label)

In [ ]:
X_test  = np.array(X_test, np.float32) / 255.

In [ ]:
preds  = model.predict(X_test, verbose = 1)

In [ ]:
submission = pd.DataFrame(preds)

In [ ]:
one_hot.columns

In [ ]:
submission.columns = one_hot.columns

In [ ]:
submission.insert(0, 'id', test_labels['id'])

In [ ]:
submission.to_csv('submission_vgg19.csv', index= False)

In [ ]:
submission.shape

In [ ]:
submission